# Keywords Excraction - Mohamed Rostom GHARBI

In [1]:
import re 
import itertools
import operator
import copy
import igraph
import heapq
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag

In [2]:
def clean_text_simple(text, my_stopwords, punct, remove_stopwords=True, pos_filtering=True, stemming=True):
    text = text.lower()
    text = ''.join(l for l in text if l not in punct) # remove punctuation (preserving intra-word dashes)
    text = re.sub(' +',' ',text) # strip extra white space
    text = text.strip() # strip leading and trailing white space
    # tokenize (split based on whitespace)
    # store results as 'tokens' #
    tokens = text.split()
    ### Split on the space
    
    if pos_filtering == True:
        # POS tag and retain only nouns and adjectives
        tagged_tokens = pos_tag(tokens)
        tokens_keep = []
        for item in tagged_tokens:
            if (
            item[1] == 'NN' or
            item[1] == 'NNS' or
            item[1] == 'NNP' or
            item[1] == 'NNPS' or
            item[1] == 'JJ' or
            item[1] == 'JJS' or
            item[1] == 'JJR'
            ):
                tokens_keep.append(item[0])
        tokens = tokens_keep
    
    if remove_stopwords:
        # remove stopwords from 'tokens'
        tokens = [token for token in tokens if token not in set(my_stopwords)]
    
    if stemming:
        # apply Porter's stemmer
        stemmer = nltk.stem.PorterStemmer()
        tokens_stemmed = list()
        for token in tokens:
            tokens_stemmed.append(stemmer.stem(token))
        tokens = tokens_stemmed
    
    return(tokens)

In [3]:
def terms_to_graph(terms, w):
    '''This function returns a directed, weighted igraph from a list of terms (the tokens from the pre-processed text) e.g., ['quick','brown','fox'].
    Edges are weighted based on term co-occurence within a sliding window of fixed size 'w'.
    '''
    
    from_to = {}
    
    # create initial complete graph (first w terms)
    terms_temp = terms[0:w]
    indexes = list(itertools.combinations(range(w), r=2))
    
    new_edges = []
    
    for my_tuple in indexes:
        new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
    
    for new_edge in new_edges:
        if new_edge in from_to:
            from_to[new_edge] += 1
        else:
            from_to[new_edge] = 1

    # then iterate over the remaining terms
    for i in range(w, len(terms)):
        considered_term = terms[i] # term to consider
        terms_temp = terms[(i-w+1):(i+1)] # all terms within sliding window
        
        # edges to try
        candidate_edges = []
        for p in range(w-1):
            candidate_edges.append((terms_temp[p],considered_term))
    
        for try_edge in candidate_edges:
            
            if try_edge[1] != try_edge[0]:
            # if not self-edge
            
                # if edge has already been seen, update its weight
                if try_edge in from_to: 
                    from_to[try_edge]+=1
                                   
                # if edge has never been seen, create it and assign it a unit weight     
                else:
                    from_to[try_edge]=1 
    
    # create empty graph
    g = igraph.Graph(directed=True)
    
    # add vertices
    g.add_vertices(sorted(set(terms)))
    
    # add edges, direction is preserved since the graph is directed
    g.add_edges(from_to.keys())
    
    # set edge and vertex weights
    g.es['weight'] = list(from_to.values()) # based on co-occurence within sliding window
    g.vs['weight'] = g.strength(weights=list(from_to.values())) # weighted degree
    
    return(g)

In [4]:
def unweighted_k_core(g):
    # work on clone of g to preserve g 
    gg = copy.deepcopy(g)    
    
    # initialize dictionary that will contain the core numbers
    cores_g = dict(zip(gg.vs['name'],[0]*len(gg.vs)))
    
    i = 0
    
    # while there are vertices remaining in the graph
    while len(gg.vs)>0:
        while [deg for deg in gg.strength() if deg<=i]:
            index = [ind for ind, deg in enumerate(gg.strength()) if deg<=i][0]
            cores_g[gg.vs[index]['name']] = i
            gg.delete_vertices(index)
        
        i += 1
    
    return cores_g

In [5]:
def weighted_core_dec(g):
    '''
    k-core decomposition for weighted graphs (generalized k-cores)
    based on Batagelj and Zaversnik's (2002) algorithm #4
    '''
    # work on clone of g to preserve g 
    gg = copy.deepcopy(g)    
    # initialize dictionary that will contain the core numbers
    cores_g = dict(zip(gg.vs["name"],[0]*len(gg.vs["name"])))

    # initialize min heap of degrees
    heap_g = zip(gg.vs["weight"],gg.vs["name"])
    heapq.heapify(list(heap_g))

    while len(list(heap_g)) > 0:
        
        top = heap_g[0][1]
        # find vertex index of heap top element
        index_top = gg.vs["name"].index(top)
        # save names of its neighbors
        neighbors_top = gg.vs[gg.neighbors(top)]["name"]
        # exclude self-edges
        neighbors_top = [elt for elt in neighbors_top if elt!=top]
        # set core number of heap top element as its weighted degree
        cores_g[top] = gg.vs[index_top]["weight"]
        # delete top vertex
        gg.delete_vertices(index_top)
        
        if len(neighbors_top)>0:
        # iterate over neighbors of top element
            for i, name_n in enumerate(neighbors_top):
                index_n = gg.vs["name"].index(name_n)
                max_n = max(cores_g[top],gg.strength(weights=gg.es["weight"])[index_n])
                gg.vs[index_n]["weight"] = max_n
                # update heap
                heap_g = zip(gg.vs["weight"],gg.vs["name"])
                heapq.heapify(heap_g)
        else:
            # update heap
            heap_g = zip(gg.vs["weight"],gg.vs["name"])
            heapq.heapify(heap_g)
            
    # sort vertices by decreasing core number
    sorted_cores_g = dict(sorted(cores_g.items(), key=operator.itemgetter(1), reverse=True))
    
    return(sorted_cores_g)

In [6]:
def accuracy_metrics(candidate, truth):
    
    # true positives ('hits') are both in candidate and in truth
    tp = len(set(candidate).intersection(truth))
    
    # false positives ('false alarms') are in candidate but not in truth
    fp = len([element for element in candidate if element not in truth])
    
    # false negatives ('misses') are in truth but not in candidate
    fn = len([element for element in truth if element not in candidate])
    
    # precision
    prec = round(float(tp)/(tp+fp),5)
    
    # recall
    rec = round(float(tp)/(tp+fn),5)
    
    if prec+rec != 0:
        # F1 score
        f1 = round(2 * float(prec*rec)/(prec+rec),5)
    else:
        f1 = 0
       
    return (prec, rec, f1)

In [7]:
import os
import random
import string
import re 
import itertools
import operator
import copy
import igraph
import nltk
from nltk.corpus import stopwords
# requires nltk >= 3.2.1
from nltk import pos_tag

# might also be required:
#They are required :
nltk.download('maxent_treebank_pos_tagger')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

# import custom functions
from library import clean_text_simple, terms_to_graph, accuracy_metrics, weighted_core_dec

stemmer = nltk.stem.PorterStemmer()
stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

##################################
# read and pre-process abstracts #
##################################

path_to_abstracts = "../data/Hulth2003testing/abstracts/"
abstract_names = sorted(os.listdir(path_to_abstracts))

abstracts = []

for counter,filename in enumerate(abstract_names):
    # read file
    with open(path_to_abstracts + filename, 'r') as my_file: 
        text = my_file.read().splitlines()
    text = ' '.join(text)
    # remove formatting
    text = re.sub('\s+', ' ', text)
    abstracts.append(text)
    # print progress
    if counter % round(len(abstract_names)/10) == 0:
        print (counter, 'files processed')

abstracts_cleaned = []
for counter,abstract in enumerate(abstracts):
    my_tokens = clean_text_simple(abstract,my_stopwords=stpwds,punct=punct)
    abstracts_cleaned.append(my_tokens)
    # print progress
    if counter % round(len(abstracts)/10) == 0:
        print(counter, 'abstracts processed')
                       
###############################################
# read and pre-process gold standard keywords #
###############################################

path_to_keywords = "../data/Hulth2003testing/uncontr/"
keyword_names = sorted(os.listdir(path_to_keywords))
   
keywords_gold_standard = []

for counter,filename in enumerate(keyword_names):
    # read file
    with open(path_to_keywords + filename, 'r') as my_file: 
        text = my_file.read().splitlines()
    text = ' '.join(text)
    text =  re.sub('\s+', ' ', text) # remove formatting
    text = text.lower()
    # turn string into list of keywords, preserving intra-word dashes 
    # but breaking n-grams into unigrams
    keywords = text.split(';')
    keywords = [keyword.strip().split(' ') for keyword in keywords]
    keywords = [keyword for sublist in keywords for keyword in sublist] # flatten list
    keywords = [keyword for keyword in keywords if keyword not in stpwds] # remove stopwords (rare but can happen due to n-gram breaking)
    keywords_stemmed = [stemmer.stem(keyword) for keyword in keywords]
    keywords_stemmed_unique = list(set(keywords_stemmed)) # remove duplicates (can happen due to n-gram breaking)
    keywords_gold_standard.append(keywords_stemmed_unique)
    
    # print progress
    if counter % round(len(keyword_names)/10) == 0:
        print(counter, 'files processed')

[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/rostom/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/rostom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0 files processed
50 files processed
100 files processed
150 files processed
200 files processed
250 files processed
300 files processed
350 files processed
400 files processed
450 files processed
0 abstracts processed
50 abstracts processed
100 abstracts processed
150 abstracts processed
200 abstracts processed
250 abstracts processed
300 abstracts processed
350 abstracts processed
400 abstracts processed
450 abstracts processed
0 files processed
50 files processed
100 files processed
150 files processed
200 files processed
250 files processed
300 files processed
350 files processed
400 files processed
450 files processed


In [8]:
###############################
# keyword extraction with gow #
###############################

#-------- gow

keywords_gow = []  

for counter,abstract in enumerate(abstracts_cleaned):
    # create graph-of-words
    g = terms_to_graph(abstract, w=4)
    # decompose graph-of-words
    core_numbers = dict(zip(g.vs['name'],g.coreness()))
    # retain main core as keywords
    max_c_n = max(core_numbers.values())
    keywords = [kwd for kwd,c_n in core_numbers.items() if c_n==max_c_n]
    # save results
    keywords_gow.append(keywords)
    
    # print progress
    if counter % round(len(abstracts_cleaned)/10) == 0:
        print(counter, 'abstracts processed')

#-------- gow_w
        
keywords_gow_w = []  


for counter,abstract in enumerate(abstracts_cleaned):
    
    # create graph-of-words
    g = terms_to_graph(abstract, w=4)
    # decompose graph-of-words
    core_numbers = weighted_core_dec(g)
    # retain main core as keywords
    max_c_n = max(core_numbers.values())
    keywords = [kwd for kwd,c_n in core_numbers.items() if c_n==max_c_n]
    # save results
    keywords_gow_w.append(keywords)
    
    # print progress
    if counter % round(len(abstracts_cleaned)/10) == 0:
        print(counter, 'abstracts processed')

#-------- TF-IDF
my_percentage = 0.33

# to ensure same pre-processing as the other methods
abstracts_cleaned_strings = [' '.join(elt) for elt in abstracts_cleaned]

tfidf_vectorizer = TfidfVectorizer(stop_words=stpwds) # is stpwds necessary here?
doc_term_matrix = tfidf_vectorizer.fit_transform(abstracts_cleaned_strings)

### create an object 'terms' containing the column names of 'doc_term_matrix' ###
### We will use the .get_feature_names() method ###
terms = tfidf_vectorizer.get_feature_names()

vectors_list = doc_term_matrix.todense().tolist()

keywords_tfidf = []

for counter,vector in enumerate(vectors_list):
    # bow feature vector as list of tuples
    terms_weights = zip(terms,vector)
    # keep only non zero values (the words in the document)
    
    nonzero = [term for term in terms_weights if term[1] != 0]
    # rank by decreasing weights
    nonzero = sorted(nonzero, key=operator.itemgetter(1), reverse=True)
    
    # retain top 'my_percentage' words as keywords
    numb_to_retain = int(len(nonzero)*my_percentage)
    
    keywords = [tuple[0] for tuple in nonzero[:numb_to_retain]]
    
    keywords_tfidf.append(keywords)
    
    # print progress
    if counter % round(len(vectors_list)/10) == 0:
        print(counter, 'vectors processed')

#-------- PageRank
keywords_pr = []

for counter,abstract in enumerate(abstracts_cleaned):
    ### fill the gaps ###
    ### hint: combine the beginning of the gow loop with the middle section of the tfidf loop ###
    # create graph-of-words
    g = terms_to_graph(abstract, w=4)
    # decompose graph-of-words
    pr_scores = zip(g.vs['name'],g.pagerank())
    pr_scores = sorted(pr_scores, key=operator.itemgetter(1), reverse=True)
    # retain main core as keywords
    numb_to_retain = int(len(pr_scores)*my_percentage)
    keywords = [tuple[0] for tuple in pr_scores[:numb_to_retain]]

    keywords_pr.append(keywords)

0 abstracts processed
50 abstracts processed
100 abstracts processed
150 abstracts processed
200 abstracts processed
250 abstracts processed
300 abstracts processed
350 abstracts processed
400 abstracts processed
450 abstracts processed
0 abstracts processed
50 abstracts processed
100 abstracts processed
150 abstracts processed
200 abstracts processed
250 abstracts processed
300 abstracts processed
350 abstracts processed
400 abstracts processed
450 abstracts processed
0 vectors processed
50 vectors processed
100 vectors processed
150 vectors processed
200 vectors processed
250 vectors processed
300 vectors processed
350 vectors processed
400 vectors processed
450 vectors processed


In [9]:
##########################
# performance evaluation #
##########################

perf_gow = [] 
perf_gow_w = []
perf_tfidf = []
perf_pr = []

for idx, truth in enumerate(keywords_gold_standard):
    perf_gow.append(accuracy_metrics(keywords_gow[idx], truth))
    perf_gow_w.append(accuracy_metrics(keywords_gow_w[idx], truth))
    perf_tfidf.append(accuracy_metrics(keywords_tfidf[idx], truth))
    perf_pr.append(accuracy_metrics(keywords_pr[idx], truth))

lkgs = len(keywords_gold_standard)


# macro-averaged results (averaged at the collection level)

results = {'gow':perf_gow,'gow_w':perf_gow_w,'tfidf':perf_tfidf,'pr':perf_pr}

for name, result in results.items():
    print(name + ' performance: \n')
    print('precision:', sum([tuple[0] for tuple in result])/lkgs)
    print('recall:', sum([tuple[1] for tuple in result])/lkgs)
    print('F-1 score:', sum([tuple[2] for tuple in result])/lkgs)
    print('\n')

gow performance: 

precision: 0.5186082200000002
recall: 0.6255520200000001
F-1 score: 0.5154552000000003


gow_w performance: 

precision: 0.4431831600000007
recall: 0.8579845000000001
F-1 score: 0.5663241799999997


tfidf performance: 

precision: 0.5920879200000003
recall: 0.3850372400000003
F-1 score: 0.4485490400000005


pr performance: 

precision: 0.6018220800000008
recall: 0.3829783200000002
F-1 score: 0.4496253800000002




In [13]:
import string
from nltk.corpus import stopwords


stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

my_doc = '''A method for solution of systems of linear algebraic equations 
with m-dimensional lambda matrices. A system of linear algebraic 
equations with m-dimensional lambda matrices is considered. 
The proposed method of searching for the solution of this system 
lies in reducing it to a numerical system of a special kind.'''

my_doc = my_doc.replace('\n', '')

# pre-process document
my_tokens = clean_text_simple(my_doc,my_stopwords=stpwds,punct=punct)
                              
g = terms_to_graph(my_tokens, w=4)
  

# decompose g
core_numbers = unweighted_k_core(g)
print(core_numbers)

print('-------------------------------------------------')

# retain main core as keywords
max_c_n = max(core_numbers.values())
keywords = [kwd for kwd, c_n in core_numbers.items() if c_n == max_c_n]
print(keywords)

{'algebra': 6, 'equat': 6, 'kind': 3, 'lambda': 6, 'linear': 6, 'm-dimension': 6, 'matric': 6, 'method': 6, 'numer': 4, 'solut': 6, 'special': 3, 'system': 6}
-------------------------------------------------
['algebra', 'equat', 'lambda', 'linear', 'm-dimension', 'matric', 'method', 'solut', 'system']
